In [2]:
# setup

import numpy as np, pickle
import matplotlib.pyplot as plt
from matplotlib import rcParams
import tensorflow as tf
from utils import *

%matplotlib inline
%load_ext autoreload
%autoreload 2

params = {
   'axes.labelsize': 16,
   'font.size': 16,
   'legend.fontsize': 16,
   'xtick.labelsize': 13,
   'ytick.labelsize': 13,
   'text.usetex': False,
    'font.family':"sans-serif",
   'font.sans-serif':'Arial',
   'text.usetex': False,
   }

#mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)
mnist = input_data.read_data_sets("data/fashion", one_hot=True)

rcParams.update(params)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Extracting data/fashion\train-images-idx3-ubyte.gz
Extracting data/fashion\train-labels-idx1-ubyte.gz
Extracting data/fashion\t10k-images-idx3-ubyte.gz
Extracting data/fashion\t10k-labels-idx1-ubyte.gz


In [ ]:
def DZ_reg2D(weights,chain=1,discount=0.9,chain_stride=1):
    result = 0
    for i in np.arange(chain):
        weights_rolled_1=tf.manip.roll(weights, shift=[-chain_stride*(i+1),-(i+1)], axis=[0,2])
        weights_rolled_2=tf.manip.roll(weights, shift=[-chain_stride*(i+1),-(i+1)], axis=[1,3])
        result += tf.reduce_mean(tf.square(weights_rolled_1-weights))*(discount**i)+tf.reduce_mean(tf.square(weights_rolled_2-weights))*(discount**i)        
    return result

In [ ]:
def fully_connected2D(pool=4, num_channels=10, verbose=True, return_weights=False, reg_kind=None, reg_value=None, chain=1, discount=0.9,stride=1):
    
    d = int(784/pool/pool)
    x = tf.placeholder(tf.float32, [None, 784])
    H_dim = 9
    #hidden layer
    W_fc1 = get_variable([28,28, H_dim,H_dim,num_channels])
    b_fc1 = get_variable([H_dim,H_dim,num_channels])
    # W_fc1 = weight_variable([784, 7840])
    # b_fc1 = bias_variable([7840])
    x_reshaped = tf.reshape(x, [-1, 28, 28, 1])
    #x_reshaped = tf.nn.max_pool(x_reshaped, [1, pool, pool,1], [1, pool, pool, 1], padding='SAME')
    #x_reshaped = tf.reshape(x_reshaped, [-1, d])
    h_fc1 = tf.sigmoid(tf.tensordot(x_reshaped, W_fc1, axes= [[1,2],[0,1]]) + b_fc1)
    h_fc1 = tf.reshape(h_fc1,[-1,H_dim*H_dim*num_channels] )
    
    #output layer
    W_fc2 = get_variable([H_dim*H_dim*num_channels, 10])
    # W_fc2 = weight_variable([7840, 10])
    b_fc2 = get_variable([10])
    y = tf.matmul(h_fc1, W_fc2) + b_fc2

    y_ = tf.placeholder(tf.float32, [None, 10])
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y))
    loss = regularize(loss, W_fc1, reg_kind, reg_value, chain, discount,stride)
    train_step = tf.train.GradientDescentOptimizer(3.0).minimize(loss)

    with tf.Session() as sess:
        tf.global_variables_initializer().run()

        correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

        for i in range(1000):
            batch = mnist.train.next_batch(100)
            train_step.run(feed_dict={x: batch[0], y_: batch[1]})
            if i%100==0:
                accuracy_train = accuracy.eval(feed_dict={x: mnist.train.images, y_: mnist.train.labels})
                accuracy_valid = accuracy.eval(feed_dict={x: mnist.test.images, y_: mnist.test.labels})
                if verbose:
                    print('Train acc:',accuracy_train, 'Valid acc:',accuracy_valid)
        
        return_values = [accuracy_train, accuracy_valid]
        if return_weights:
            return_values.append(sess.run(W_fc1))
        
    return return_values